In [6]:
project_id = !gcloud config list --format 'value(core.project)' | head -n 1
project_id = project_id[0]
project_id

'alekseyv-gke'

In [ ]:
from absl import app
from absl import flags
import sys
import time
import os
import numpy

import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

DATASET_GCP_PROJECT_ID = "bigquery-public-data"
DATASET_ID = "samples"
TABLE_ID = "wikipedia"
requested_streams = 2
sloppy = True
batch_size = 32

client = BigQueryClient()
read_session = client.read_session(
  "projects/" + project_id,
  DATASET_GCP_PROJECT_ID, TABLE_ID, DATASET_ID,
  ["title",
   "id",
   "num_characters",
   "language",
   "timestamp",
   "wp_namespace",
   "contributor_username"],
  [dtypes.string,
   dtypes.int64,
   dtypes.int64,
   dtypes.string,
   dtypes.int64,
   dtypes.int64,
   dtypes.string],
  requested_streams=requested_streams
  )

streams = read_session.get_streams()
print('Requested %d streams, BigQuery returned %d streams' % (
    requested_streams
    len(streams)))
dataset = read_session.parallel_read_rows(sloppy=sloppy).batch(batch_size)
row_index = 0
for row in dataset.prefetch(10).take(10):
    print("row %d: %s" % (row_index, row))
    row_index += 1



In [ ]:
from absl import app
from absl import flags
import sys
import time
import os
import numpy

import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

DATASET_GCP_PROJECT_ID = "bigquery-public-data"
DATASET_ID = "samples"
TABLE_ID = "wikipedia"
requested_streams = 2
sloppy = True
batch_size = 32

client = BigQueryClient()
read_session = client.read_session(
  "projects/" + project_id,
  DATASET_GCP_PROJECT_ID, TABLE_ID, DATASET_ID,
  ["title",
   "id",
   "num_characters",
   "language",
   "timestamp",
   "wp_namespace",
   "contributor_username"],
  [dtypes.string,
   dtypes.int64,
   dtypes.int64,
   dtypes.string,
   dtypes.int64,
   dtypes.int64,
   dtypes.string],
  requested_streams=requested_streams
  )

streams = read_session.get_streams()
print('Requested %d streams, BigQuery returned %d streams' % (
    requested_streams,
    len(streams)))

def read_rows(stream):
    dataset = read_session.read_rows(stream)
    dataset = dataset.batch(batch_size)
    return dataset

streams = read_session.get_streams()
streams_count=len(streams)
streams_ds = tf.data.Dataset.from_tensor_slices(streams)
dataset = streams_ds.interleave(
    read_rows,
    cycle_length=streams_count,
    num_parallel_calls=streams_count,
    deterministic=False)

row_index = 0
for row in dataset.prefetch(10).take(10):
    print("row %d: %s" % (row_index, row))
    row_index += 1



In [ ]:
!python sample.py --project_id=$project_id

In [ ]:
!docker build --build-arg BASE_IMAGE=tensorflow/tensorflow:2.5.0 --build-arg TF_IO=tensorflow-io==0.19.1 -f Dockerfile -t tf2.5 . && docker run -v ${PWD}:/v tf2.3 python3 /v/sample.py --project_id='alekseyv-gke'


In [ ]:
!docker build --build-arg BASE_IMAGE=tensorflow/tensorflow:2.3.0 --build-arg TF_IO=tensorflow-io==0.15.0 -f Dockerfile -t tf2.3 . && docker run -v ${PWD}:/v tf2.3 python3 /v/bq_perftest_mult_columns.py --project_id=alekseyv-gke --batch_size=2048 --num_iterations=100 --mini_batch_size=10 --num_columns=120 --requested_streams=1 --sloppy=true --format=AVRO
